In [149]:
#from google.colab import drive
#drive.mount('/content/drive')

In [150]:
import csv
import numpy as np
import pandas as pd

In [151]:
#converting Jan2013 features csv file into numpy array
with open('/content/drive/MyDrive/features-2013-1.csv','r') as dest_f:
    next(dest_f) #omitting the header from file before converting it into array
    data_iter = csv.reader(dest_f, delimiter = ",", quotechar = '"')
    data = [data for data in data_iter]
data_array = np.asarray(data)

In [152]:
data_array.shape

(6217, 9)

In [153]:
data_array # full array

array([['1', '1dice6YgEVBf88erBFra9BHf6ZMoyvG88', '1000000', ..., '1',
        '0', '0'],
       ['1', '133f33VEDKrVTwtQYgh5z4rLLhP9ibm6tk', '3036250', ..., '1',
        '0', '0'],
       ['1', '1DvMt7dGAZeKqzJF4nDx74qK37ziftRjTf', '4086250', ..., '1',
        '0', '0'],
       ...,
       ['31', '1JRuyQHmrtgq4etXREPjsuEccVLLSAhNCy', '1008564450', ...,
        '1', '0', '0'],
       ['31', '1BMYCbkUYoLvPGg1CsnSN2XG77yshGCiUw', '1906341761', ...,
        '1', '0', '0'],
       ['31', '12gpoiDCeWyV72BQqiWfUYsf7PbZ5vDkcY', '1823248525', ...,
        '1', '0', '0']], dtype='<U34')

In [154]:
data_array_bckp = np.copy(data_array)

In [155]:
data_array_bckp.shape

(6217, 9)

In [156]:
type(data_array)

numpy.ndarray

In [157]:
#function which take a DAY number as input, 
#and returns 2 arrays: 1st array =  rows of 3 consecutive days AND 2nd array: rows of 4th day
#1st array is used for training and 2nd array is used for testing
def day_slide (d1):
    day1 = str(d1)
    day2 = str(d1 + 1)
    day3 = str(d1 + 2)
    day4 = str(d1 + 3)

    rows_index1=np.where(data_array[:,0]== day1 )
    array_d1 = data_array[rows_index1]

    rows_index2=np.where(data_array[:,0]== day2 )
    array_d2 = data_array[rows_index2]

    rows_index3=np.where(data_array[:,0]== day3 )
    array_d3 = data_array[rows_index3]

    rows_index4=np.where(data_array[:,0]== day4 )
    array_d4 = data_array[rows_index4]

    array_of_3days = np.vstack((array_d1,array_d2,array_d3))

    return array_of_3days,array_d4

In [158]:
#dropping address column
filtered_data_array = np.delete(data_array, 1, axis=1) # 3rd param axis=1 ==> col, axis=0 ==> row ; 2nd param is the index

In [159]:
filtered_data_array.shape

(6217, 8)

In [160]:
data_array = np.copy(filtered_data_array)

In [161]:
data_array_bkup2 = np.copy(data_array.astype(int))

In [162]:
#norm = np.linalg.norm(data_array)
#data_array = data_array[:,1:3]/norm

In [163]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
xgb = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.6,
                      subsample = 1,
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=6, 
                      gamma=1)

In [186]:
#process of above example needs to be put in loop for all DAYS
# looping through each DAY of Jan2013
# window size = 4 ( 3 days for train, 4th day for pred)

sum_accu = 0
accu_list = []

for i in range(1,int(data_array[-1,0])+1-3): # last day in the dataset --> int(data_array[-1,0]) # here 31 days - 3 days
    trainset, testset = day_slide(i)
    print("\nday",i)
    x_train = trainset[:,:6].astype(int) # if we donot want to include the 'day' column, then use [1:,:6]
    y_train = trainset[:,7].astype(int)
    #y_train = trainset[:,7].astype(int).reshape(-1,1)
    x_test = testset[:,:6].astype(int)
    y_test = testset[:,7].astype(int)
    #y_test = testset[:,7].astype(int).reshape(-1,1)


    #trying to combine all the train data till this point(not just the 3 days), for model fitting
    if i == 1:
        final_x_train = np.copy(x_train)
        final_y_train = np.copy(y_train)
    else:                       
        final_x_train = np.vstack((final_x_train,x_train))
        #final_x_train = np.unique(final_x_train2,axis=0)
        final_y_train = np.hstack((final_y_train,y_train))
        #final_y_train = np.unique(final_y_train2,axis=0)

    xgb.fit(final_x_train, final_y_train)
    xgb_pred = xgb.predict(x_test)
    curr_accu = accuracy_score(y_test, xgb_pred)
    print("current accuracy =",curr_accu)
    print ("y_test\n",y_test)
    print ("y_pred\n",xgb_pred)
    accu_list.append(curr_accu) #list of series accuracies for each loop, to observe the inc or dec 
    sum_accu += curr_accu
    #mean_accu = (sum_accu)/i

    if i == int(data_array[-1,0])+1-3 :
        final_accuracy = curr_accu
        mean_accu = (sum_accu)/i


    # model needs to be trained and tested uing the above trainset and testset everytime in the loop
    # model needs to be saved and loaded everytime, so that fitting doesnot override past learning/experience of the model
    # need to decide on the model to be used for prediction



day 1
current accuracy = 0.9950248756218906
y_test
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
y_pred
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

day 2
current accuracy = 0.9900990099009901
y_test
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [178]:
curr_accu #final_accuracy

1.0

In [179]:
mean_accu

27.83699371083007

In [180]:
accu_list

[0.9950248756218906,
 0.9900990099009901,
 0.9950248756218906,
 0.9900990099009901,
 1.0,
 0.9950248756218906,
 1.0,
 0.9900990099009901,
 1.0,
 1.0,
 0.9950248756218906,
 0.9950248756218906,
 1.0,
 0.9852216748768473,
 0.9852216748768473,
 1.0,
 0.9950248756218906,
 0.9900990099009901,
 1.0,
 0.9950248756218906,
 0.9950248756218906,
 0.9950248756218906,
 1.0,
 1.0,
 0.975609756097561,
 0.9900990099009901,
 0.9852216748768473,
 1.0]

In [188]:
#process of above example needs to be put in loop for all DAYS
# looping through each DAY of Jan2013
# window size = 4 ( 3 days for train, 4th day for pred)

sum_accu = 0
accu_list = []

for i in range(1,int(data_array[-1,0])+1-3): # last day in the dataset --> int(data_array[-1,0]) # here 31 days - 3 days
    trainset, testset = day_slide(i)
    print("\nday",i)
    x_train = trainset[:,:6].astype(int) # if we donot want to include the 'day' column, then use [1:,:6]
    y_train = trainset[:,7].astype(int)
    #y_train = trainset[:,7].astype(int).reshape(-1,1)
    x_test = testset[:,:6].astype(int)
    y_test = testset[:,7].astype(int)
    #y_test = testset[:,7].astype(int).reshape(-1,1)

    ###omitting this part of code makes the loop run faster, But now the data of just the 3 days is used for training
          #trying to combine all the train data till this point, for model fitting
          #if i == 1:
              #final_x_train = np.copy(x_train)
              #final_y_train = np.copy(y_train)
          #else:                       
              #final_x_train = np.vstack((final_x_train,x_train))
              ##final_x_train = np.unique(final_x_train2,axis=0)
              #final_y_train = np.hstack((final_y_train,y_train))
              ##final_y_train = np.unique(final_y_train2,axis=0)
    ###

    xgb.fit(x_train, y_train)
    xgb_pred = xgb.predict(x_test)
    curr_accu = accuracy_score(y_test, xgb_pred)
    print("current accuracy =",curr_accu)
    print ("y_test\n",y_test)
    print ("y_pred\n",xgb_pred)
    accu_list.append(curr_accu) #list of series accuracies for each loop, to observe the inc or dec 
    sum_accu += curr_accu
    #mean_accu = (sum_accu)/i

    if i == int(data_array[-1,0])+1-3 :
        final_accuracy = curr_accu
        mean_accu = (sum_accu)/i


    # model needs to be trained and tested uing the above trainset and testset everytime in the loop
    # model needs to be saved and loaded everytime, so that fitting doesnot override past learning/experience of the model
    # need to improve train dataset on the model to be used for prediction


day 1
current accuracy = 0.9950248756218906
y_test
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
y_pred
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

day 2
current accuracy = 0.9900990099009901
y_test
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [189]:
mean_accu

27.83699371083007

In [190]:
accu_list

[0.9950248756218906,
 0.9900990099009901,
 0.9950248756218906,
 0.9900990099009901,
 1.0,
 0.9950248756218906,
 1.0,
 0.9900990099009901,
 1.0,
 1.0,
 0.9950248756218906,
 0.9950248756218906,
 1.0,
 0.9852216748768473,
 0.9852216748768473,
 1.0,
 0.9950248756218906,
 0.9900990099009901,
 1.0,
 0.9950248756218906,
 0.9950248756218906,
 0.9950248756218906,
 1.0,
 1.0,
 0.975609756097561,
 0.9900990099009901,
 0.9852216748768473,
 1.0]